In [ ]:
import pandas as pd 
import numpy as np 
import sklearn 
import nltk 
import re
import string 
import nltk 
from nltk.corpus import stopwords 

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score 
from sklearn.metrics import accuracy_score, f1_score, precision_score


In [ ]:
phrases = [' I want to learn machine learning from Innomatics Research Labs', " i am working at Innomatics Research Labs"]

In [ ]:
vect = CountVectorizer()
vect.fit(phrases)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [ ]:
print("vocab size", len(vect.vocabulary_))

vocab size 12


In [ ]:
print(vect.vocabulary_)

{'want': 10, 'to': 9, 'learn': 5, 'machine': 7, 'learning': 6, 'from': 2, 'innomatics': 3, 'research': 8, 'labs': 4, 'am': 0, 'working': 11, 'at': 1}


In [ ]:
bag_of_words = vect.transform(phrases)

In [ ]:
print(bag_of_words)

  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (1, 0)	1
  (1, 1)	1
  (1, 3)	1
  (1, 4)	1
  (1, 8)	1
  (1, 11)	1


In [ ]:
print('bag of words', bag_of_words.toarray())

bag of words [[0 0 1 1 1 1 1 1 1 1 1 0]
 [1 1 0 1 1 0 0 0 1 0 0 1]]


In [ ]:
dataset = pd.read_csv(r"/content/drive/MyDrive/IMDB Dataset.csv")
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset['sentiment'] = dataset['sentiment'].map({'positive':1, 'negative':0})

In [ ]:
dataset.__len__()

50000

In [ ]:
word_count  =  [nltk.word_tokenize(dataset.review[i]).__len__() for i in range(0, len(dataset.review))]

In [ ]:
print(word_count.__len__())

50000


In [ ]:
word_count[1]

201

In [ ]:
print(sum(word_count))

13956873


In [ ]:
def cleaning_reviews(df):
    all_reviews = list()
    lines = df['review'].values.tolist()
    for i in lines:
        text = i.lower()
        pattern = re.compile('http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:[0-9a-fA-F][0-9a-fA-F]))+')
        text = pattern.sub("", text)
        tokens = nltk.word_tokenize(text)
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        stop_word = set(stopwords.words('english'))
        stop_word.discard("not")
        words = [nltk.WordNetLemmatizer().lemmatize(w) for  w in words if not w in stop_word]
        words = ' '.join(words)
        all_reviews.append(words)
    return(all_reviews)

In [ ]:
vals = cleaning_reviews(dataset)

In [ ]:
print(vals[0])

one reviewer mentioned watching oz episode hooked right exactly happened br br first thing struck oz brutality unflinching scene violence set right word go trust not show faint hearted timid show pull punch regard drug sex violence hardcore classic use word br br called oz nickname given oswald maximum security state penitentary focus mainly emerald city experimental section prison cell glass front face inwards privacy not high agenda em city home manyaryans muslim gangsta latino christian italian irish scuffle death stare dodgy dealing shady agreement never far away br br would say main appeal show due fact go show would nt dare forget pretty picture painted mainstream audience forget charm forget romance oz nt mess around first episode ever saw struck nasty surreal could nt say ready watched developed taste oz got accustomed high level graphic violence not violence injustice crooked guard sold nickel inmate kill order get away well mannered middle class inmate turned prison bitch due

In [ ]:
words_count_after_cleaning  = [nltk.word_tokenize(vals[i]).__len__() for i in range(0, len(vals))]
print(sum(words_count_after_cleaning))

6168646


In [ ]:
vector = CountVectorizer(max_features=5000)
x= vector.fit_transform(vals).toarray()
y = dataset['sentiment'].to_numpy()
print(x.shape)
print(y.shape)

(50000, 5000)
(50000,)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=42)
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
print("Accuracy score", accuracy_score(Y_test, Y_pred))
print("precision Score", precision_score(Y_test, Y_pred))

Accuracy score 0.739
precision Score 0.8142302716688228
